<h1>Imports</h1>

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
from nltk.corpus import stopwords
import emoji as em
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder

<h1>Import des données</h1>

In [ ]:
data_train = pd.read_xml("./data/apprentissage/train.xml")
data_train = data_train.dropna()

data_dev = pd.read_xml("./data/apprentissage/dev.xml")
data_dev = data_dev.dropna()

data_test = pd.read_xml("./data/test/test.xml")
data_test = data_test.dropna()

<h1>Prédictions en prénant en compte les emoticons : Approche 1</h1>

In [ ]:
emoticons = pd.read_csv("./data/processed/emoticons.csv", sep=' ')

def get_near_class(note):
    if note<2.0:
        return "negative"
    elif note<=3.0:
        return "neutral"
    else:
        return "positive"

emoticons["near_class"] = emoticons["note_mean"].apply(lambda x: get_near_class(x))
emoticons = emoticons.sort_values(by=["count"], ascending=False)
print(emoticons)

In [ ]:
temp = data_train
temp2 = data_dev

tempp = {"negative": -1, "neutral": 0, "positive": 1}

def get_emoticon_score(text):
    score = 0
    found = False
    for i in text:
        if i in em.EMOJI_DATA:
            isin = emoticons[emoticons["emoji"]==i]
            if len(isin)==1:
                found = True
                score = score + tempp[isin["near_class"].to_list()[0]]
    if not found:
        return "not data"
    else:
        if score>=1:
            return "positive"
        elif score==0:
            return "neutral"
        else:
            return "negative"
            
temp["emoticon"] = temp["commentaire"].apply(lambda x: get_emoticon_score(x))
temp2["emoticon"] = temp2["commentaire"].apply(lambda x: get_emoticon_score(x))

In [ ]:
temp[temp["emoticon"].str.contains("not data")==False]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

tfidf = TfidfVectorizer(
    stop_words = stopwords.words("french") + stopwords.words("english"),
    analyzer = 'word',
    lowercase = True,
    ngram_range = (1,2)
)
preprocessor = ColumnTransformer(transformers=[
                ('tfidf', tfidf, "commentaire"),
                ("scaler", OneHotEncoder(), ['emoticon'])
            ])
model = make_pipeline(preprocessor, SGDClassifier(loss = "hinge", penalty = "l2"))
model.fit(temp, temp["note"])

In [ ]:
y_pred = model.predict(temp2)
accuracy_score(data_dev["note"], y_pred)

<h1>Prédictions en prénant en compte les emoticons : Approche 2</h1>

In [ ]:
emoticons = pd.read_csv("./data/externe/emojisentiment.csv", sep=',')
maxxx = emoticons[["Negative","Neutral","Positive"]].idxmax(axis=1).to_frame()
maxxx["Emoji"] = emoticons["Emoji"]
emoticons = maxxx.rename(columns={0: "sentiment", "Emoji": "emoji"})

emoticons = dict(zip(emoticons["emoji"].to_list(), emoticons["sentiment"].to_list()))

temp = data_train
temp2 = data_dev

tempp = {"Negative": -1, "Neutral": 0, "Positive": 1}

def get_emoticon_score(text):
    score = 0
    found = False
    for i in text:
        if i in emoticons.keys():
            found = True
            score = score + tempp[emoticons[i]]
    if not found:
        return "not data"
    else:
        if score>=1:
            return "positive"
        elif score==0:
            return "neutral"
        else:
            return "negative"
    
temp["emoticon"] = temp["commentaire"].apply(lambda x: get_emoticon_score(x))
temp2["emoticon"] = temp2["commentaire"].apply(lambda x: get_emoticon_score(x))

In [ ]:
temp[temp["emoticon"].str.contains("not data")==False]

In [ ]:
tfidf = TfidfVectorizer(
    stop_words = stopwords.words("french"),
    analyzer = 'word',
    lowercase = True,
    ngram_range = (1,2)
)
preprocessor = ColumnTransformer(transformers=[
                ('tfidf', tfidf, "commentaire"),
                ("scaler", OneHotEncoder(), ['emoticon'])
            ])
model = make_pipeline(preprocessor, SGDClassifier(loss="hinge", penalty = "l2"))
model.fit(temp, temp["note"])

In [ ]:
y_pred = model.predict(temp2)
accuracy_score(data_dev["note"], y_pred)